In [1]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn import metrics
import torch.optim as optim

In [2]:
sub_features = ['42 tGravityAcc-mean()-Y',
 '43 tGravityAcc-mean()-Z',
 '51 tGravityAcc-max()-Y',
 '52 tGravityAcc-max()-Z',
 '54 tGravityAcc-min()-Y',
 '55 tGravityAcc-min()-Z',
 '56 tGravityAcc-sma()',
 '58 tGravityAcc-energy()-Y',
 '59 tGravityAcc-energy()-Z',
 '128 tBodyGyro-mad()-Y',
 '141 tBodyGyro-iqr()-Y',
 '428 fBodyGyro-std()-Y',
 '434 fBodyGyro-max()-Y',
 '475 fBodyGyro-bandsEnergy()-1,8',
 '483 fBodyGyro-bandsEnergy()-1,16',
 '487 fBodyGyro-bandsEnergy()-1,24',
 '559 angle(X,gravityMean)',
 '560 angle(Y,gravityMean)',
 '561 angle(Z,gravityMean)']

act_features = ['4 tBodyAcc-std()-X',
 '7 tBodyAcc-mad()-X',
 '10 tBodyAcc-max()-X',
 '17 tBodyAcc-energy()-X',
 '202 tBodyAccMag-std()',
 '203 tBodyAccMag-mad()',
 '215 tGravityAccMag-std()',
 '216 tGravityAccMag-mad()',
 '266 fBodyAcc-mean()-X',
 '269 fBodyAcc-std()-X',
 '282 fBodyAcc-energy()-X',
 '303 fBodyAcc-bandsEnergy()-1,8',
 '311 fBodyAcc-bandsEnergy()-1,16',
 '315 fBodyAcc-bandsEnergy()-1,24',
 '382 fBodyAccJerk-bandsEnergy()-1,8',
 '504 fBodyAccMag-std()',
 '505 fBodyAccMag-mad()',
 '509 fBodyAccMag-energy()']

input_shape = len(sub_features) + len(act_features)

In [3]:
def classifier_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.LeakyReLU(0.05)
    )

class Classifier(nn.Module):
    def __init__(self, feature_dim = input_shape):
        super(Classifier, self).__init__()
        self.network = nn.Sequential(
            classifier_block(feature_dim, 30),
            classifier_block(30, 25),
            classifier_block(25, 20),
            classifier_block(20, 15),
            nn.Linear(15, 12)
        )
    def forward(self, x):
        return self.network(x)

In [4]:
#defines each generator layer
#input and output dimensions needed
def generator_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.BatchNorm1d(output_dim),
        nn.ReLU(inplace = True)
    )

#returns n_samples of z_dim (number of dimensions of latent space) noise
def get_noise(n_samples, z_dim):
    return torch.randn(n_samples, z_dim)

#defines generator class
class Generator(nn.Module):
    def __init__(self, z_dim = 10, feature_dim = input_shape, hidden_dim = 128):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            generator_block(z_dim, 80),
            generator_block(80, 60),
            generator_block(60, 50),
            nn.Linear(50, feature_dim),
            nn.Tanh()
        )
    def forward(self, noise):
        return self.gen(noise)

def load_model(model, model_name):
    model.load_state_dict(torch.load(f'../../../saved_models/{model_name}'))

In [5]:
#label is a list of integers specifying which labels to filter by
#users is a list of integers specifying which users to filter by
#y_label is a string, either "Activity" or "Subject" depending on what y output needs to be returned
def start_data(label, users, y_label, sub_features, act_features):
    #get the dataframe column names
    name_dataframe = pd.read_csv('../../../data/features.txt', delimiter = '\n', header = None)
    names = name_dataframe.values.tolist()
    names = [k for row in names for k in row] #List of column names

    data = pd.read_csv('../../../data/X_train.txt', delim_whitespace = True, header = None) #Read in dataframe
    data.columns = names #Setting column names
    
    X_train_1 = data[sub_features]
    X_train_2 = data[act_features]
    X_train = pd.concat([X_train_1, X_train_2], axis = 1)
    
    y_train_activity = pd.read_csv('../../../data/y_train.txt', header = None)
    y_train_activity.columns = ['Activity']
    
    y_train_subject = pd.read_csv('../../../data/subject_train.txt', header = None)
    y_train_subject.columns = ['Subject']
    
    GAN_data = pd.concat([X_train, y_train_activity, y_train_subject], axis = 1)
    GAN_data = GAN_data[GAN_data['Activity'].isin(label)]
    GAN_data = GAN_data[GAN_data['Subject'].isin(users)]
    
    X_1 = GAN_data[(GAN_data['Subject'] == 1) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_2 = GAN_data[(GAN_data['Subject'] == 1) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_3 = GAN_data[(GAN_data['Subject'] == 1) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    X_4 = GAN_data[(GAN_data['Subject'] == 3) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_5 = GAN_data[(GAN_data['Subject'] == 3) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_6 = GAN_data[(GAN_data['Subject'] == 3) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    X_7 = GAN_data[(GAN_data['Subject'] == 5) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_8 = GAN_data[(GAN_data['Subject'] == 5) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_9 = GAN_data[(GAN_data['Subject'] == 5) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    X_10 = GAN_data[(GAN_data['Subject'] == 7) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_11 = GAN_data[(GAN_data['Subject'] == 7) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_12 = GAN_data[(GAN_data['Subject'] == 7) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    
    X_train = np.concatenate((X_1, X_2, X_3, X_4, X_5, X_6, X_7, X_8, X_9, X_10, X_11, X_12))
    y_train = [0] * len(X_1) + [1] * len(X_2) + [2] * len(X_3) + [3] * len(X_4) + [4] * len(X_5) + [5] * len(X_6) + [6] * len(X_7) + [7] * len(X_8) + [8] * len(X_9) + [9] * len(X_10) + [10] * len(X_11) + [11] * len(X_12)
    
    return X_train, np.asarray(y_train)

In [6]:
activities = [1, 3, 4]
users = [1, 3, 5, 7]

X, y = start_data(activities, users, "Activity", sub_features, act_features)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = True)

model = Classifier()
lr = 0.001
n_epochs = 5000
batch_size = 250

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = lr)

train_features = torch.tensor(X_train)
train_labels = torch.tensor(y_train)
test_features = torch.tensor(X_test)
test_labels = torch.tensor(y_test)

train_data = torch.utils.data.TensorDataset(train_features, train_labels)
test_data = torch.utils.data.TensorDataset(test_features, test_labels)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = len(test_labels), shuffle = True)

In [7]:
for epoch in range(n_epochs):
    total_loss = 0
    for batch in train_loader:
        features, labels = batch
        
        optimizer.zero_grad()
        preds = model(features.float())
        
        loss = criterion(preds, labels.long()) 
        loss.backward()
        
        optimizer.step()
        total_loss += loss.item()
        
    print(f'Epoch {epoch + 1}, Loss: {total_loss}, Final Batch Loss: {loss.item()}')

Epoch 1, Loss: 7.422689199447632, Final Batch Loss: 2.453273057937622
Epoch 2, Loss: 7.370434284210205, Final Batch Loss: 2.406439781188965
Epoch 3, Loss: 7.4301841259002686, Final Batch Loss: 2.477592706680298
Epoch 4, Loss: 7.389699459075928, Final Batch Loss: 2.442140579223633
Epoch 5, Loss: 7.482944965362549, Final Batch Loss: 2.5409023761749268
Epoch 6, Loss: 7.366950750350952, Final Batch Loss: 2.422630786895752
Epoch 7, Loss: 7.390000820159912, Final Batch Loss: 2.4611198902130127
Epoch 8, Loss: 7.3953564167022705, Final Batch Loss: 2.4677884578704834
Epoch 9, Loss: 7.428051710128784, Final Batch Loss: 2.5073204040527344
Epoch 10, Loss: 7.333410739898682, Final Batch Loss: 2.425410509109497
Epoch 11, Loss: 7.342511892318726, Final Batch Loss: 2.442734479904175
Epoch 12, Loss: 7.303533554077148, Final Batch Loss: 2.4173624515533447
Epoch 13, Loss: 7.3596882820129395, Final Batch Loss: 2.481877326965332
Epoch 14, Loss: 7.262422561645508, Final Batch Loss: 2.404369354248047
Epoch 1

Epoch 117, Loss: 3.427583336830139, Final Batch Loss: 1.1823242902755737
Epoch 118, Loss: 3.2597157955169678, Final Batch Loss: 1.063146710395813
Epoch 119, Loss: 3.4876513481140137, Final Batch Loss: 1.3867236375808716
Epoch 120, Loss: 3.2262483835220337, Final Batch Loss: 1.0066908597946167
Epoch 121, Loss: 3.1837856769561768, Final Batch Loss: 1.025815725326538
Epoch 122, Loss: 3.4219497442245483, Final Batch Loss: 1.3123468160629272
Epoch 123, Loss: 3.139937400817871, Final Batch Loss: 0.9784226417541504
Epoch 124, Loss: 3.1385037899017334, Final Batch Loss: 0.9966672658920288
Epoch 125, Loss: 3.267960786819458, Final Batch Loss: 1.139164924621582
Epoch 126, Loss: 3.433777928352356, Final Batch Loss: 1.2146730422973633
Epoch 127, Loss: 3.080708682537079, Final Batch Loss: 0.9566335082054138
Epoch 128, Loss: 3.121229887008667, Final Batch Loss: 1.026881456375122
Epoch 129, Loss: 3.207728147506714, Final Batch Loss: 1.1390676498413086
Epoch 130, Loss: 3.0669084191322327, Final Batch 

Epoch 240, Loss: 2.562880039215088, Final Batch Loss: 0.9492486715316772
Epoch 241, Loss: 2.4186298847198486, Final Batch Loss: 0.7012877464294434
Epoch 242, Loss: 2.501971125602722, Final Batch Loss: 0.9094704985618591
Epoch 243, Loss: 2.660338878631592, Final Batch Loss: 1.071550726890564
Epoch 244, Loss: 2.515383720397949, Final Batch Loss: 0.9008351564407349
Epoch 245, Loss: 2.4574268460273743, Final Batch Loss: 0.7946351170539856
Epoch 246, Loss: 2.9826626777648926, Final Batch Loss: 1.2346558570861816
Epoch 247, Loss: 2.659198820590973, Final Batch Loss: 0.9632810354232788
Epoch 248, Loss: 2.4432127475738525, Final Batch Loss: 0.781440258026123
Epoch 249, Loss: 2.620486855506897, Final Batch Loss: 1.0203454494476318
Epoch 250, Loss: 2.398397982120514, Final Batch Loss: 0.7455203533172607
Epoch 251, Loss: 2.658624231815338, Final Batch Loss: 1.0219988822937012
Epoch 252, Loss: 2.513192296028137, Final Batch Loss: 0.8394344449043274
Epoch 253, Loss: 2.2527737617492676, Final Batch 

Epoch 356, Loss: 2.417973220348358, Final Batch Loss: 1.0592554807662964
Epoch 357, Loss: 2.0828887820243835, Final Batch Loss: 0.7447357177734375
Epoch 358, Loss: 1.9610441327095032, Final Batch Loss: 0.5549768805503845
Epoch 359, Loss: 2.0557820200920105, Final Batch Loss: 0.6312133073806763
Epoch 360, Loss: 2.0889267325401306, Final Batch Loss: 0.6124946475028992
Epoch 361, Loss: 2.090624511241913, Final Batch Loss: 0.688174307346344
Epoch 362, Loss: 2.152690351009369, Final Batch Loss: 0.6995115280151367
Epoch 363, Loss: 1.762330025434494, Final Batch Loss: 0.3550420105457306
Epoch 364, Loss: 2.1940632462501526, Final Batch Loss: 0.8724821209907532
Epoch 365, Loss: 1.9057292342185974, Final Batch Loss: 0.5616421103477478
Epoch 366, Loss: 2.27535480260849, Final Batch Loss: 0.8767470121383667
Epoch 367, Loss: 2.237858235836029, Final Batch Loss: 0.7914186120033264
Epoch 368, Loss: 2.1019963026046753, Final Batch Loss: 0.6712498068809509
Epoch 369, Loss: 1.8404226005077362, Final Bat

Epoch 482, Loss: 2.0395464301109314, Final Batch Loss: 0.7394116520881653
Epoch 483, Loss: 1.8157429099082947, Final Batch Loss: 0.594957172870636
Epoch 484, Loss: 1.8317511081695557, Final Batch Loss: 0.6016479730606079
Epoch 485, Loss: 2.142364203929901, Final Batch Loss: 0.8083497881889343
Epoch 486, Loss: 1.917198657989502, Final Batch Loss: 0.6853881478309631
Epoch 487, Loss: 1.8265911936759949, Final Batch Loss: 0.660183310508728
Epoch 488, Loss: 2.219134509563446, Final Batch Loss: 0.9851030111312866
Epoch 489, Loss: 1.8089088797569275, Final Batch Loss: 0.5741105079650879
Epoch 490, Loss: 1.896019160747528, Final Batch Loss: 0.6555838584899902
Epoch 491, Loss: 2.1121652722358704, Final Batch Loss: 0.9024850726127625
Epoch 492, Loss: 1.6129043996334076, Final Batch Loss: 0.42457345128059387
Epoch 493, Loss: 1.6560169160366058, Final Batch Loss: 0.4028647840023041
Epoch 494, Loss: 2.494306445121765, Final Batch Loss: 1.186206340789795
Epoch 495, Loss: 1.9452713131904602, Final Ba

...

Epoch 4468, Loss: 0.41845228895545006, Final Batch Loss: 0.046510692685842514
Epoch 4469, Loss: 0.5482931733131409, Final Batch Loss: 0.1403018832206726
Epoch 4470, Loss: 0.5372059643268585, Final Batch Loss: 0.17591188848018646
Epoch 4471, Loss: 0.42055535316467285, Final Batch Loss: 0.07252363860607147
Epoch 4472, Loss: 0.4643116518855095, Final Batch Loss: 0.046049974858760834
Epoch 4473, Loss: 0.5849869102239609, Final Batch Loss: 0.2092042714357376
Epoch 4474, Loss: 0.49206380173563957, Final Batch Loss: 0.03200076147913933
Epoch 4475, Loss: 0.562303215265274, Final Batch Loss: 0.18300510942935944
Epoch 4476, Loss: 0.5737955719232559, Final Batch Loss: 0.2175697237253189
Epoch 4477, Loss: 0.47572020441293716, Final Batch Loss: 0.11665342003107071
Epoch 4478, Loss: 0.5000303760170937, Final Batch Loss: 0.0731649324297905
Epoch 4479, Loss: 0.45450038462877274, Final Batch Loss: 0.06895139068365097
Epoch 4480, Loss: 0.44563233852386475, Final Batch Loss: 0.1088365912437439
Epoch 4481

Epoch 4591, Loss: 0.4722956568002701, Final Batch Loss: 0.09191827476024628
Epoch 4592, Loss: 0.5601796656847, Final Batch Loss: 0.1453900784254074
Epoch 4593, Loss: 0.43695785105228424, Final Batch Loss: 0.06572481989860535
Epoch 4594, Loss: 0.3598969653248787, Final Batch Loss: 0.048959411680698395
Epoch 4595, Loss: 0.8736111223697662, Final Batch Loss: 0.521377444267273
Epoch 4596, Loss: 0.4448918104171753, Final Batch Loss: 0.1390306055545807
Epoch 4597, Loss: 0.4267287403345108, Final Batch Loss: 0.09687879681587219
Epoch 4598, Loss: 0.6089490205049515, Final Batch Loss: 0.21086464822292328
Epoch 4599, Loss: 0.5652153193950653, Final Batch Loss: 0.14944550395011902
Epoch 4600, Loss: 0.4155363067984581, Final Batch Loss: 0.06861060112714767
Epoch 4601, Loss: 0.6443134546279907, Final Batch Loss: 0.2819133996963501
Epoch 4602, Loss: 0.5695138871669769, Final Batch Loss: 0.2505939304828644
Epoch 4603, Loss: 0.590709775686264, Final Batch Loss: 0.226286381483078
Epoch 4604, Loss: 0.60

Epoch 4705, Loss: 0.802967980504036, Final Batch Loss: 0.4389391839504242
Epoch 4706, Loss: 0.537649855017662, Final Batch Loss: 0.14576134085655212
Epoch 4707, Loss: 0.39133111014962196, Final Batch Loss: 0.04225415363907814
Epoch 4708, Loss: 0.4885893240571022, Final Batch Loss: 0.0969734713435173
Epoch 4709, Loss: 0.4436807781457901, Final Batch Loss: 0.057528287172317505
Epoch 4710, Loss: 0.6094308644533157, Final Batch Loss: 0.2046162486076355
Epoch 4711, Loss: 0.5493947714567184, Final Batch Loss: 0.059905439615249634
Epoch 4712, Loss: 0.7897484302520752, Final Batch Loss: 0.3597220778465271
Epoch 4713, Loss: 0.48472652584314346, Final Batch Loss: 0.06436029821634293
Epoch 4714, Loss: 0.6612119078636169, Final Batch Loss: 0.26062408089637756
Epoch 4715, Loss: 0.5471373349428177, Final Batch Loss: 0.09767836332321167
Epoch 4716, Loss: 0.5774001479148865, Final Batch Loss: 0.19543695449829102
Epoch 4717, Loss: 0.5505374521017075, Final Batch Loss: 0.15709568560123444
Epoch 4718, Lo

Epoch 4824, Loss: 0.4595402777194977, Final Batch Loss: 0.14068925380706787
Epoch 4825, Loss: 0.5080630928277969, Final Batch Loss: 0.1394677758216858
Epoch 4826, Loss: 0.5646183490753174, Final Batch Loss: 0.1541563719511032
Epoch 4827, Loss: 0.6777383238077164, Final Batch Loss: 0.289362370967865
Epoch 4828, Loss: 0.7357814759016037, Final Batch Loss: 0.4109489321708679
Epoch 4829, Loss: 0.4957343861460686, Final Batch Loss: 0.07853015512228012
Epoch 4830, Loss: 0.7675891071557999, Final Batch Loss: 0.4001816213130951
Epoch 4831, Loss: 0.49940426647663116, Final Batch Loss: 0.11924347281455994
Epoch 4832, Loss: 0.4174622744321823, Final Batch Loss: 0.06989549100399017
Epoch 4833, Loss: 0.5525332689285278, Final Batch Loss: 0.16172771155834198
Epoch 4834, Loss: 0.5762249380350113, Final Batch Loss: 0.2138686180114746
Epoch 4835, Loss: 0.42181286960840225, Final Batch Loss: 0.09344453364610672
Epoch 4836, Loss: 0.5289512276649475, Final Batch Loss: 0.1526632457971573
Epoch 4837, Loss: 

Epoch 4938, Loss: 0.4067825600504875, Final Batch Loss: 0.07966230064630508
Epoch 4939, Loss: 0.3703414648771286, Final Batch Loss: 0.03475138545036316
Epoch 4940, Loss: 0.5314431488513947, Final Batch Loss: 0.16947665810585022
Epoch 4941, Loss: 0.5673286020755768, Final Batch Loss: 0.27035948634147644
Epoch 4942, Loss: 0.7594276368618011, Final Batch Loss: 0.4586440622806549
Epoch 4943, Loss: 0.36691969633102417, Final Batch Loss: 0.07721082866191864
Epoch 4944, Loss: 0.38941009156405926, Final Batch Loss: 0.02073451690375805
Epoch 4945, Loss: 0.3872009553015232, Final Batch Loss: 0.05404757335782051
Epoch 4946, Loss: 0.4176507517695427, Final Batch Loss: 0.1159445121884346
Epoch 4947, Loss: 0.5686311274766922, Final Batch Loss: 0.22443921864032745
Epoch 4948, Loss: 0.44788189977407455, Final Batch Loss: 0.04361575096845627
Epoch 4949, Loss: 0.5025646686553955, Final Batch Loss: 0.16353854537010193
Epoch 4950, Loss: 0.49135328084230423, Final Batch Loss: 0.11711966246366501
Epoch 4951

In [8]:
softmax = nn.Softmax(dim = 1)
model.eval()
for batch in test_loader:
    features, labels = batch
    _, preds = torch.max(softmax(model(features.float())), dim = 1)
    print(metrics.confusion_matrix((labels).cpu(), preds.cpu()))
    print(metrics.classification_report((labels).cpu(), preds.cpu(), digits = 5))

[[19  0  0  0  0  0  0  0  0  0  0  0]
 [ 0 12  0  0  0  0  0  0  0  0  0  0]
 [ 0  0 12  0  0  0  0  0  0  0  0  0]
 [ 0  0  0 12  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  8  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  5  0  0  1  0  0  0]
 [ 0  0  0  0  0  0 11  0  0  0  0  0]
 [ 0  0  0  0  0  0  0 11  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  6  0  0  2]
 [ 0  0  0  0  0  0  0  0  0 12  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  8  0]
 [ 0  0  0  0  0  0  0  0  0  0  0 11]]
              precision    recall  f1-score   support

           0    1.00000   1.00000   1.00000        19
           1    1.00000   1.00000   1.00000        12
           2    1.00000   1.00000   1.00000        12
           3    1.00000   1.00000   1.00000        12
           4    1.00000   1.00000   1.00000         8
           5    1.00000   0.83333   0.90909         6
           6    1.00000   1.00000   1.00000        11
           7    1.00000   1.00000   1.00000        11
           8    0.85714   0.75000   0.80000 

In [23]:
n_samples = 20

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U0A0 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_1 = gen(latent_vectors).detach().numpy()

y_1 = np.zeros(n_samples)

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U0A1 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_2 = gen(latent_vectors).detach().numpy()

y_2 = np.ones(n_samples)

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U0A2 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_3 = gen(latent_vectors).detach().numpy()

y_3 = np.ones(n_samples) + 1

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U1A0 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_4 = gen(latent_vectors).detach().numpy()

y_4 = np.ones(n_samples) + 2

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U1A1 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_5 = gen(latent_vectors).detach().numpy()

y_5 = np.ones(n_samples) + 3

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U1A2 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_6 = gen(latent_vectors).detach().numpy()

y_6 = np.ones(n_samples) + 4

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U2A0 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_7 = gen(latent_vectors).detach().numpy()

y_7 = np.ones(n_samples) + 5

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U2A1 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_8 = gen(latent_vectors).detach().numpy()

y_8 = np.ones(n_samples) + 6

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U2A2 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_9 = gen(latent_vectors).detach().numpy()

y_9 = np.ones(n_samples) + 7

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U3A0 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_10 = gen(latent_vectors).detach().numpy()

y_10 = np.ones(n_samples) + 8

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U3A1 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_11 = gen(latent_vectors).detach().numpy()

y_11 = np.ones(n_samples) + 9

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U3A2 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_12 = gen(latent_vectors).detach().numpy()

y_12 = np.ones(n_samples) + 10

fake_features = np.concatenate((fake_features_1, fake_features_2, fake_features_3, fake_features_4, fake_features_5, fake_features_6,
                         fake_features_7, fake_features_8, fake_features_9, fake_features_10, fake_features_11, fake_features_12))
fake_labels = np.concatenate((y_1, y_2, y_3, y_4, y_5, y_6, y_7, y_8, y_9, y_10, y_11, y_12))

fake_features = torch.Tensor(fake_features)

In [24]:
_, preds = torch.max(softmax(model(fake_features.float())), dim = 1)
print(metrics.confusion_matrix((fake_labels), preds.cpu()))
print(metrics.classification_report((fake_labels), preds.cpu(), digits = 5))

[[19  0  0  0  0  0  0  0  0  1  0  0]
 [ 1 19  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  9  0  0  6  0  0  5  0  0  0]
 [ 0  0  0 20  0  0  0  0  0  0  0  0]
 [ 0  0  0  0 15  0  2  0  0  0  3  0]
 [ 0  0  3  0  0  8  0  0  8  0  0  1]
 [ 0  0  0  0  0  0 20  0  0  0  0  0]
 [ 0  0  0  0  0  0  0 20  0  0  0  0]
 [ 0  0  0  0  0  4  0  0 10  4  0  2]
 [ 0  0  0  0  1  0  7  0  0  8  4  0]
 [ 0  0  0  0  2  0  0  0  0  0 18  0]
 [ 0  0  0  0  0 14  0  0  2  0  0  4]]
              precision    recall  f1-score   support

         0.0    0.95000   0.95000   0.95000        20
         1.0    1.00000   0.95000   0.97436        20
         2.0    0.75000   0.45000   0.56250        20
         3.0    1.00000   1.00000   1.00000        20
         4.0    0.83333   0.75000   0.78947        20
         5.0    0.25000   0.40000   0.30769        20
         6.0    0.68966   1.00000   0.81633        20
         7.0    1.00000   1.00000   1.00000        20
         8.0    0.40000   0.50000   0.44444 